In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from segmodel import *
from new_segmodel import,non_local_network
from loss import *
from dataloader import *
import torch
import numpy as np
import time
from timeit import default_timer as timer
import torch.nn.functional as F
from rate import *
from torch import optim
from torchcontrib.optim import SWA
from tqdm import tqdm
import cv2
import lovasz_losses as L
torch.cuda.set_device(0)
from random import shuffle
from tqdm import tqdm_notebook
#from HRNet import seg_hrnet
#from non_local_model import asymmetric_non_local_network

TkAgg


C:\Users\hyc\Desktop\胸部超声数据集\Dataset_BUSI\new_segmodel.py:305: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.weight, 0)
C:\Users\hyc\Desktop\胸部超声数据集\Dataset_BUSI\new_segmodel.py:306: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(self.W.bias, 0)


Total: 40237625 Trainable: 40237625


In [2]:
skip = ["encoder.layer0.bn1.num_batches_tracked", "encoder.layer1.0.bn1.num_batches_tracked",
                "encoder.layer1.0.bn2.num_batches_tracked", "encoder.layer1.0.bn3.num_batches_tracked",
                "encoder.layer1.0.downsample.1.num_batches_tracked", "encoder.layer1.1.bn1.num_batches_tracked",
                "encoder.layer1.1.bn2.num_batches_tracked", "encoder.layer1.1.bn3.num_batches_tracked",
                "encoder.layer1.2.bn1.num_batches_tracked", "encoder.layer1.2.bn2.num_batches_tracked",
                "encoder.layer1.2.bn3.num_batches_tracked", "encoder.layer2.0.bn1.num_batches_tracked",
                "encoder.layer2.0.bn2.num_batches_tracked", "encoder.layer2.0.bn3.num_batches_tracked",
                "encoder.layer2.0.downsample.1.num_batches_tracked", "encoder.layer2.1.bn1.num_batches_tracked",
                "encoder.layer2.1.bn2.num_batches_tracked", "encoder.layer2.1.bn3.num_batches_tracked",
                "encoder.layer2.2.bn1.num_batches_tracked", "encoder.layer2.2.bn2.num_batches_tracked",
                "encoder.layer2.2.bn3.num_batches_tracked", "encoder.layer2.3.bn1.num_batches_tracked",
                "encoder.layer2.3.bn2.num_batches_tracked", "encoder.layer2.3.bn3.num_batches_tracked",
                "encoder.layer3.0.bn1.num_batches_tracked", "encoder.layer3.0.bn2.num_batches_tracked",
                "encoder.layer3.0.bn3.num_batches_tracked", "encoder.layer3.0.downsample.1.num_batches_tracked",
                "encoder.layer3.1.bn1.num_batches_tracked", "encoder.layer3.1.bn2.num_batches_tracked",
                "encoder.layer3.1.bn3.num_batches_tracked", "encoder.layer3.2.bn1.num_batches_tracked",
                "encoder.layer3.2.bn2.num_batches_tracked", "encoder.layer3.2.bn3.num_batches_tracked",
                "encoder.layer3.3.bn1.num_batches_tracked", "encoder.layer3.3.bn2.num_batches_tracked",
                "encoder.layer3.3.bn3.num_batches_tracked", "encoder.layer3.4.bn1.num_batches_tracked",
                "encoder.layer3.4.bn2.num_batches_tracked", "encoder.layer3.4.bn3.num_batches_tracked",
                "encoder.layer3.5.bn1.num_batches_tracked", "encoder.layer3.5.bn2.num_batches_tracked",
                "encoder.layer3.5.bn3.num_batches_tracked", "encoder.layer4.0.bn1.num_batches_tracked",
                "encoder.layer4.0.bn2.num_batches_tracked", "encoder.layer4.0.bn3.num_batches_tracked",
                "encoder.layer4.0.downsample.1.num_batches_tracked", "encoder.layer4.1.bn1.num_batches_tracked",
                "encoder.layer4.1.bn2.num_batches_tracked", "encoder.layer4.1.bn3.num_batches_tracked",
                "encoder.layer4.2.bn1.num_batches_tracked", "encoder.layer4.2.bn2.num_batches_tracked",
                "encoder.layer4.2.bn3.num_batches_tracked", "encoder1.bn1.num_batches_tracked",
                "encoder2.0.bn1.num_batches_tracked", "encoder2.0.bn2.num_batches_tracked",
                "encoder2.0.bn3.num_batches_tracked", "encoder2.0.downsample.1.num_batches_tracked",
                "encoder2.1.bn1.num_batches_tracked", "encoder2.1.bn2.num_batches_tracked",
                "encoder2.1.bn3.num_batches_tracked", "encoder2.2.bn1.num_batches_tracked",
                "encoder2.2.bn2.num_batches_tracked", "encoder2.2.bn3.num_batches_tracked",
                "encoder3.0.bn1.num_batches_tracked", "encoder3.0.bn2.num_batches_tracked",
                "encoder3.0.bn3.num_batches_tracked", "encoder3.0.downsample.1.num_batches_tracked",
                "encoder3.1.bn1.num_batches_tracked", "encoder3.1.bn2.num_batches_tracked",
                "encoder3.1.bn3.num_batches_tracked", "encoder3.2.bn1.num_batches_tracked",
                "encoder3.2.bn2.num_batches_tracked", "encoder3.2.bn3.num_batches_tracked",
                "encoder3.3.bn1.num_batches_tracked", "encoder3.3.bn2.num_batches_tracked",
                "encoder3.3.bn3.num_batches_tracked", "encoder4.0.bn1.num_batches_tracked",
                "encoder4.0.bn2.num_batches_tracked", "encoder4.0.bn3.num_batches_tracked",
                "encoder4.0.downsample.1.num_batches_tracked", "encoder4.1.bn1.num_batches_tracked",
                "encoder4.1.bn2.num_batches_tracked", "encoder4.1.bn3.num_batches_tracked",
                "encoder4.2.bn1.num_batches_tracked", "encoder4.2.bn2.num_batches_tracked",
                "encoder4.2.bn3.num_batches_tracked", "encoder4.3.bn1.num_batches_tracked",
                "encoder4.3.bn2.num_batches_tracked", "encoder4.3.bn3.num_batches_tracked",
                "encoder4.4.bn1.num_batches_tracked", "encoder4.4.bn2.num_batches_tracked",
                "encoder4.4.bn3.num_batches_tracked", "encoder4.5.bn1.num_batches_tracked",
                "encoder4.5.bn2.num_batches_tracked", "encoder4.5.bn3.num_batches_tracked",
                "encoder5.0.bn1.num_batches_tracked", "encoder5.0.bn2.num_batches_tracked",
                "encoder5.0.bn3.num_batches_tracked", "encoder5.0.downsample.1.num_batches_tracked",
                "encoder5.1.bn1.num_batches_tracked", "encoder5.1.bn2.num_batches_tracked",
                "encoder5.1.bn3.num_batches_tracked", "encoder5.2.bn1.num_batches_tracked",
                "encoder5.2.bn2.num_batches_tracked", "encoder5.2.bn3.num_batches_tracked",
                "center.0.bn.num_batches_tracked", "center.2.bn.num_batches_tracked",
                "decoder5.conv1.bn.num_batches_tracked", "decoder5.conv2.bn.num_batches_tracked",
                "decoder5.context.0.context.1.num_batches_tracked",
                "decoder5.context.0.context.3.stages.0.f_key.1.num_batches_tracked",
                "decoder5.context.0.context.3.stages.0.f_query.1.num_batches_tracked",
                "decoder5.context.0.context.3.conv_bn_dropout.1.num_batches_tracked",
                "decoder5.context.0.conv2.1.num_batches_tracked", "decoder5.context.0.conv3.1.num_batches_tracked",
                "decoder5.context.0.conv4.1.num_batches_tracked", "decoder5.context.0.conv5.1.num_batches_tracked",
                "decoder5.context.0.conv_bn_dropout.1.num_batches_tracked", "decoder4.conv1.bn.num_batches_tracked",
                "decoder4.conv2.bn.num_batches_tracked", "decoder4.context.0.context.1.num_batches_tracked",
                "decoder4.context.0.context.3.stages.0.f_key.1.num_batches_tracked",
                "decoder4.context.0.context.3.stages.0.f_query.1.num_batches_tracked",
                "decoder4.context.0.context.3.conv_bn_dropout.1.num_batches_tracked",
                "decoder4.context.0.conv2.1.num_batches_tracked", "decoder4.context.0.conv3.1.num_batches_tracked",
                "decoder4.context.0.conv4.1.num_batches_tracked", "decoder4.context.0.conv5.1.num_batches_tracked",
                "decoder4.context.0.conv_bn_dropout.1.num_batches_tracked", "decoder3.conv1.bn.num_batches_tracked",
                "decoder3.conv2.bn.num_batches_tracked", "decoder3.context.0.context.1.num_batches_tracked",
                "decoder3.context.0.context.3.stages.0.f_key.1.num_batches_tracked",
                "decoder3.context.0.context.3.stages.0.f_query.1.num_batches_tracked",
                "decoder3.context.0.context.3.conv_bn_dropout.1.num_batches_tracked",
                "decoder3.context.0.conv2.1.num_batches_tracked", "decoder3.context.0.conv3.1.num_batches_tracked",
                "decoder3.context.0.conv4.1.num_batches_tracked", "decoder3.context.0.conv5.1.num_batches_tracked",
                "decoder3.context.0.conv_bn_dropout.1.num_batches_tracked", "decoder2.conv1.bn.num_batches_tracked",
                "decoder2.conv2.bn.num_batches_tracked", "decoder2.context.0.context.1.num_batches_tracked",
                "decoder2.context.0.context.3.stages.0.f_key.1.num_batches_tracked",
                "decoder2.context.0.context.3.stages.0.f_query.1.num_batches_tracked",
                "decoder2.context.0.context.3.conv_bn_dropout.1.num_batches_tracked",
                "decoder2.context.0.conv2.1.num_batches_tracked", "decoder2.context.0.conv3.1.num_batches_tracked",
                "decoder2.context.0.conv4.1.num_batches_tracked", "decoder2.context.0.conv5.1.num_batches_tracked",
                "decoder2.context.0.conv_bn_dropout.1.num_batches_tracked", "decoder1.conv1.bn.num_batches_tracked",
                "decoder1.conv2.bn.num_batches_tracked", "decoder1.context.0.context.1.num_batches_tracked",
                "decoder1.context.0.context.3.stages.0.f_key.1.num_batches_tracked",
                "decoder1.context.0.context.3.stages.0.f_query.1.num_batches_tracked",
                "decoder1.context.0.context.3.conv_bn_dropout.1.num_batches_tracked",
                "decoder1.context.0.conv2.1.num_batches_tracked", "decoder1.context.0.conv3.1.num_batches_tracked",
                "decoder1.context.0.conv4.1.num_batches_tracked", "decoder1.context.0.conv5.1.num_batches_tracked",
                "decoder1.context.0.conv_bn_dropout.1.num_batches_tracked"]

In [3]:
def load_old_pretrain_file(net, pretrain_file, skip=[]):
    pretrain_state_dict = torch.load(pretrain_file)
    state_dict = net.state_dict()
    keys = list(state_dict.keys())
    for key in keys:
        if any(s in key for s in skip):
            continue
        if  key.startswith('conv1'):
        #if 'conv1.' in key:
            key0 = key.replace('conv1.','layer0.')
            state_dict[key] = pretrain_state_dict[key0]
            continue
        if 'encoder2.' in key:
            key0 = key.replace('encoder2.','layer1.')
            state_dict[key] = pretrain_state_dict[key0]
            continue
        if 'encoder3.' in key:
            key0 = key.replace('encoder3.','layer2.')
            state_dict[key] = pretrain_state_dict[key0]
            continue
        if 'encoder4.' in key:
            key0 = key.replace('encoder4.','layer3.')
            state_dict[key] = pretrain_state_dict[key0]
            continue
        if 'encoder5.' in key:
            key0 = key.replace('encoder5.','layer4.')
            state_dict[key] = pretrain_state_dict[key0]
            continue
    net.load_state_dict(state_dict)
    return net

def load_pretrain_file(net, pretrain_file, skip=[]):

    pretrain_state_dict = torch.load(pretrain_file)
    state_dict = net.state_dict()
    keys = list(state_dict.keys())
    for key in keys:
        if any(s in key for s in skip):
            continue

        state_dict[key] = pretrain_state_dict[key]
    net.load_state_dict(state_dict)
    return net

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from segmodel import *
from new_segmodel import SeResNeXt50Unet,MiniSeResNeXt50Unet
from loss import *
from dataloader import *
import torch
import numpy as np
import time
from timeit import default_timer as timer
import torch.nn.functional as F
from rate import *
from torch import optim
from torchcontrib.optim import SWA
from tqdm import tqdm
import cv2
import lovasz_losses as L
torch.cuda.set_device(0)
from util import EPE, getEdge
from random import shuffle
from tqdm import tqdm_notebook
#from torchkeras import summary

def load_old_pretrain_file(net, pretrain_file, skip=[]):

    pretrain_state_dict = torch.load(pretrain_file)
    state_dict = net.state_dict()
    keys = list(state_dict.keys())
    for key in keys:
        if any(s in key for s in skip):
            continue

        if  key.startswith('conv1'):
        #if 'conv1.' in key:
            key0 = key.replace('conv1.','layer0.')
            state_dict[key] = pretrain_state_dict[key0]
            continue

        if 'encoder2.' in key:
            key0 = key.replace('encoder2.','layer1.')
            state_dict[key] = pretrain_state_dict[key0]
            continue

        if 'encoder3.' in key:
            key0 = key.replace('encoder3.','layer2.')
            state_dict[key] = pretrain_state_dict[key0]
            continue

        if 'encoder4.' in key:
            key0 = key.replace('encoder4.','layer3.')
            state_dict[key] = pretrain_state_dict[key0]
            continue

        if 'encoder5.' in key:
            key0 = key.replace('encoder5.','layer4.')
            state_dict[key] = pretrain_state_dict[key0]
            continue

        #print(key)
        #state_dict[key] = pretrain_state_dict[key]
    net.load_state_dict(state_dict)
    return net

def load_pretrain_file(net, pretrain_file, skip=[]):

    pretrain_state_dict = torch.load(pretrain_file)
    state_dict = net.state_dict()
    keys = list(state_dict.keys())
    for key in keys:
        if any(s in key for s in skip):
            continue

        state_dict[key] = pretrain_state_dict[key]
    net.load_state_dict(state_dict)
    return net


################## training ###########################################
def do_valid( net, batch_size, valid_loader ):

    valid_num  = 0
    valid_loss = np.zeros(3,np.float32)

    predicts = []
    truths   = []

    for input, truth in valid_loader:
        input = input.cuda()
        truth = truth.cuda()
        with torch.no_grad():
            logit = net(input)
            prob  = F.sigmoid(logit)
            loss  = net.criterion(logit, truth)
            dice  = net.metric(logit, truth)

        #batch_size
        valid_loss += batch_size*np.array(( loss.item(), dice.item(), 0))
        valid_num += batch_size

        predicts.append(prob.data.cpu().numpy())
        truths.append(truth.data.cpu().numpy())

    #assert(valid_num == len(valid_loader.sampler))
    valid_loss  = valid_loss/valid_num

    return valid_loss

def run_train():

    out_dir = './weight_file/'
    initial_checkpoint = None
    pretrain_file = None

    ## setup-----------------------
    os.makedirs(out_dir +'/checkpoint/',exist_ok=True)
    os.makedirs(out_dir +'/train/', exist_ok=True)
    os.makedirs(out_dir +'/backup/',exist_ok=True)

    log = Logger()
    log.open(out_dir + '/log.train.txt', mode='w')
    log.write('\n--- [START %s] %s\n\n' % ('IDENTIFIER', '-' * 64))
    log.write('\t__file__     = %s\n' % __file__)
    log.write('\tout_dir      = %s\n' % out_dir)
    log.write('\n')
    log.write('\t<additional comments> ...  \n')
    log.write('\t  - 5-scale unet  \n')
    log.write('\t  - more augmentation \n')
    log.write('\n')


    train_image_dir = './train_img/'
    train_mask_dir = './train_gt/'
    train_file_list = os.listdir(train_image_dir)
    val_image_dir = './val_img/'
    val_mask_dir = './val_gt/'
    val_file_list = os.listdir(val_image_dir)

    ## dataset ----------------------------------------
    log.write('** dataset setting **\n')
    
    shape = (224,224)
    
    batch_size = 8
    
    number_workers = 0
    
    train_skindataset = SkinDataset(image_dir=train_image_dir,
                              mask_dir=train_mask_dir,
                              file_list=train_file_list,
                              shape=shape,is_test=False)

    train_dataloader = DataLoader(
        dataset=train_skindataset,
        batch_size=batch_size,
        num_workers = number_workers,
        pin_memory=True,
        shuffle=True)

    val_skindataset = SkinDataset(image_dir=val_image_dir,
                              mask_dir=val_mask_dir,
                              file_list=val_file_list,
                              shape=shape,is_test=False)
    

    val_dataloader = DataLoader(
        dataset=val_skindataset,
        batch_size=batch_size,
        num_workers = number_workers,
        pin_memory=True,
        shuffle=True)

    assert (len(train_skindataset) >= batch_size)
    log.write('batch_size = %d\n' % (batch_size))
    log.write('\n')

    #net ------------------------------------
    log.write('** net settint **\n')
    net = SaltNet().cuda()
    net.load_pretrain('./se_resnext50_32x4d-a260b3a4.pth')
    #print(net)
    if initial_checkpoint is not None:
        log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
        net.load_state_dict(torch.load(initial_checkpoint, map_location=lambda storage, loc: storage))

    if pretrain_file is not None:
        log.write('\tpretrain_file = %s\n' % pretrain_file)
        net = load_pretrain_file(net, pretrain_file, skip=['feature', 'logit'])

    log.write('%s\n' % (type(net)))
    log.write('\n')

    ## optimizer -------------------------------
    num_iters = 300 * 1000
    iter_smooth = 20
    iter_log = 50
    iter_valid = 100
    iter_save = [0, num_iters-1] \
        +list(range(0,num_iters,500)) # 1*1000

    #-----------------------------------------
    if 0: ##freeze
        for p in net.feature_net.parameters():
            p.requires_grad = False


    schduler = None  #LR = StepLR([ (0, 0.01),  (200, 0.001),  (300, -1)])
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()),
                          lr=0.005, momentum=0.9, weight_decay=0.0001)

    #opt = SWA(optmizer)
    start_iter = 0
    start_epoch= 0

    if initial_checkpoint is not None:
        checkpoint  = torch.load(initial_checkpoint.replace('_model.pth','_optimizer.pth'))
        start_iter  = checkpoint['iter' ]
        start_epoch = checkpoint['epoch']

        rate = get_learning_rate(optimizer)  #load all except learning rate
        #optimizer.load_state_dict(checkpoint['optimizer'])
        adjust_learning_rate(optimizer, rate)
        pass


    ## start training here! ##############################################
    log.write('** start training here! **\n')

    #log.write(' samples_per_epoch = %d\n\n'%len(train_dataset))
    log.write(' rate    iter   epoch   | valid_loss               | train_loss               | batch_loss               |  time          \n')
    log.write('-------------------------------------------------------------------------------------------------------------------------------\n')

    train_loss = np.zeros(6,np.float32)
    valid_loss = np.zeros(6,np.float32)
    batch_loss = np.zeros(6,np.float32)
    rate = 0
    iter = 0
    i    = 0

    start = timer()

    while iter<num_iters:

        sum_train_loss = np.zeros(6,np.float32)
        sum = 0
        optimizer.zero_grad()

        for image, mask in train_dataloader:
            if 0: #debug  ##--------------------------------------------------------
                image = image.cpu().data.numpy().squeeze()
                mask = mask.cpu().data.numpy().squeeze()

                batch_size = len(index)
                #print(image.shape)
                #print(mask.shape)

                #for b in range(batch_size):
                ##---------------------------------------

            len_train_dataset = len(train_dataloader)
            iter = i + start_iter
            epoch = (iter - start)*batch_size/len_train_dataset + start_epoch
            num_samples = epoch*len_train_dataset

            if iter % iter_valid == 0:
                net.set_mode('valid')
                val_loss = do_valid(net,batch_size,val_dataloader)
                net.set_mode('train')

                print('\r',end='',flush=True)
                log.write('%0.4f  %5.1f  %6.1f  |  %0.3f  %0.3f  (%0.3f) |  %0.3f  %0.3f  |  %0.3f  %0.3f  | %s \n' % ( \
                    rate, iter / 1000, epoch,
                    valid_loss[0], valid_loss[1], valid_loss[2],
                    train_loss[0], train_loss[1],
                    batch_loss[0], batch_loss[1],
                    str((timer() - start)/60)+ 'min'))
                time.sleep(0.01)

            if iter in iter_save:
                torch.save(net.state_dict(),out_dir +'/checkpoint/%08d_model.pth'%(iter))
                torch.save({
                    'optimizer': optimizer.state_dict(),
                    'iter'     : iter,
                    'epoch'    : epoch,
                }, out_dir +'/checkpoint/%08d_optimizer.pth'%(iter))
                pass

            #one iteration update ----------------------
            #net.seg_mode('train',is_freeze_bn=True)

            net.set_mode('train')
            image = image.cuda()
            mask  = mask.cuda()

            logit = net(image)
            loss  = net.criterion(logit, mask)
            dice  = net.metric(logit, mask)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            #torch.nn.utils.clip_grad_norm(net.parameters(),1)

            # print statistics ----------------
            batch_loss = np.array((
                loss.item(),
                dice.item(),0,0,0,0,))

            sum_train_loss += batch_loss
            sum += 1
            if iter%iter_smooth == 0:
                train_loss = sum_train_loss/sum
                sum_train_loss = np.zeros(6,np.float32)
                sum = 0


            print('\r%0.4f  %5.1f  %6.1f  |  %0.3f  %0.3f  (%0.3f) |  %0.3f  %0.3f  |  %0.3f  %0.3f  | %s ' % (\
                         rate, iter/1000, epoch,
                         valid_loss[0], valid_loss[1], valid_loss[2],
                         train_loss[0], train_loss[1],
                         batch_loss[0], batch_loss[1],
                         str((timer() - start)/60)+'min'), end='',flush=True)
            i=i+1


            #<debug> ===================================================================
            if 0:
            #if iter%200==0:
                #voxel, aux, query, link, truth, cache = make_valid_batch(valid_dataset.dataset, batch_size=2)

                net.set_mode('test')#
                with torch.no_grad():
                    logit = net(input)
                    prob  = F.sigmoid(logit)
                    loss  = net.criterion(logit, truth)
                    dice  = net.metric(logit, truth)

                    if 0:
                        loss  = net.criterion(logit, truth)
                        accuracy,hit_rate,precision_rate = net.metric(logit, truth)
                        valid_loss[0] = loss.item()
                        valid_loss[1] = accuracy.item()
                        valid_loss[2] = hit_rate.item()
                        valid_loss[3] = precision_rate.item()



                #show only b in batch ---
                b = 1
                prob   = prob.data.cpu().numpy()[b].squeeze()
                truth  = truth.data.cpu().numpy()[b].squeeze()
                input  = input.data.cpu().numpy()[b].squeeze()

                all = np.hstack([input,truth,prob])
                image_show_norm('all',all,max=1,resize=3)
                cv2.waitKey(100)

                net.set_mode('train')
            #<debug> ===================================================================


        pass  #-- end of one data loader --
    pass #-- end of all iterations --


    if 1: #save last
        torch.save(net.state_dict(),out_dir +'/checkpoint/%d_model.pth'%(i))
        torch.save({
            'optimizer': optimizer.state_dict(),
            'iter'     : i,
            'epoch'    : epoch,
        }, out_dir +'/checkpoint/%d_optimizer.pth'%(i))

    log.write('\n')
    log.close()


class Logger:

    def __int__(self):
        super(Logger, self).__int__()

    def open(self,name,mode):
        self.txt = open(name,mode=mode)

    def write(self,str_):
        self.txt.write(str_)
        print(str_)

    def close(self):
        self.txt.close()

def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_list_from_txt(txt_dir):
    img_list = []

    with open(txt_dir) as f:
        for i in f:
            img_list.append(i.strip('\n').strip('\t'))
    
    return img_list


In [7]:
import os 
from random import shuffle
data_dir = 'D:/ultrasound image/胸部超声数据集/Dataset_BUSI/Dataset_BUSI_with_GT/total/'
total_image_list  = [] 

for name in os.listdir(data_dir):
    if 'mask' in name: # remove the ground truth name
        continue
    total_image_list.append(name)

shuffle(total_image_list)
total_image_num = len(total_image_list)
train_image_num = 562 # about 0.7 * total_image_num
val_image_num   = 86  # about 0.1 * total_image_num
test_image_num  = 132 # about 0.2 * total_image_num

train_image_list = total_image_list[:train_image_num]
val_image_list   = total_image_list[train_image_num:train_image_num+val_image_num]
test_image_list  = total_image_list[train_image_num+val_image_num:]



In [7]:
total_image_dir = './Dataset_BUSI_with_GT/total/'
num_class = 1
number_workers = 2
#epochs = 20
thresh = 0.5
shape = (224,224)
batch_size = 8
multi_class = False
if num_class > 1:
    multi_class = True
#Building Dasaset

train_skindataset = SkinDataset(image_dir=total_image_dir,
                          mask_dir=total_image_dir,
                          file_list=train_image_list+train_image_list,
                          shape=shape,is_test=False,num_class=num_class,is_mask=False)


train_dataloader = DataLoader(
    dataset=train_skindataset,
    batch_size=batch_size,
    num_workers = number_workers,
    pin_memory=True,
    shuffle=True)

val_skindataset = SkinDataset(image_dir=total_image_dir,
                          mask_dir=total_image_dir,
                          file_list=val_image_list,
                          shape=shape,is_test=True,num_class=num_class,is_mask=False)

val_dataloader = DataLoader(
    dataset=val_skindataset,
    batch_size=batch_size,
    num_workers = number_workers,
    pin_memory=True,
    shuffle=True)


In [11]:
def get_parameter_number(net):
    total_num = sum(p.numel() for p in net.parameters())
    trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print('Total: {}'.format(total_num), 'Trainable: {}'.format(trainable_num))

In [12]:
def input_mixup(input1, input2, lambda1):
    lambda2 = torch.from_numpy(np.array(1.)) - lambda1
    mixup_input = input1.mul(lambda1) + input2.mul(lambda2)
    return mixup_input

def make_one_hot(labels):
	target = torch.eye(1)[labels]
	gt_1_hot = target.permute(0, 3, 1, 2).float().to(0)
	return gt_1_hot

def label_mixup(labels1, labels2, lambda1):
	lambda2 = torch.from_numpy(np.array(1.))-lambda1
	mixup_label = labels1.mul(lambda1)+ labels2.mul(lambda2)

	mixup_label_margin = 0.4
	mixup_label = (mixup_label > mixup_label_margin) * 1
	return mixup_label.float()

In [13]:
def run_my_train(out_dir,shape,swa_epoch,pretrained,weight_file,batch_size,epochs,
                 model_name,edge_model,num_class,dataset,NFN_model,SKN_module,kernel_three):
    #out_dir = './weight_file_1/'
    initial_checkpoint = None
    pretrain_file = None

    ## setup-----------------------
    os.makedirs(out_dir +'/checkpoint/',exist_ok=True)
    os.makedirs(out_dir +'/train/', exist_ok=True)
    os.makedirs(out_dir +'/backup/',exist_ok=True)

    log = Logger()
    log.open(out_dir + '/log.train_skinlesion.txt', mode='w')
    log.write('\n--- [START %s] %s\n\n' % ('IDENTIFIER', '-' * 64))
    #log.write('\t__file__     = %s\n' % __file__)
    log.write('\tout_dir      = %s\n' % out_dir)
    log.write('\n')
    log.write('\t<additional comments> ...  \n')
    log.write('\t  - 5-scale unet  \n')
    log.write('\t  - more augmentation \n')
    log.write('\n')

    total_image_dir = './Dataset_BUSI_with_GT/total/'
    if dataset == 'new':
        total_image_dir = './Dataset_BUSI_with_GT/total_new/'
    
    print(total_image_dir)
    number_workers = 2

    thresh = 0.5
    
    multi_class = False
    if num_class > 1:
        multi_class = True
    #Building Dasaset
    
    train_skindataset = SkinDataset(image_dir=total_image_dir,
                              mask_dir=total_image_dir,
                              file_list=train_image_list,
                              shape=shape,is_test=False,num_class=num_class,is_mask=False)

    
    train_dataloader = DataLoader(
        dataset=train_skindataset,
        batch_size=batch_size,
        num_workers = number_workers,
        pin_memory=True,
        shuffle=True)
   
    val_skindataset = SkinDataset(image_dir=total_image_dir,
                              mask_dir=total_image_dir,
                              file_list=val_image_list,
                              shape=shape,is_test=True,num_class=num_class,is_mask=False)

    val_dataloader = DataLoader(
        dataset=val_skindataset,
        batch_size=batch_size,
        num_workers = number_workers,
        pin_memory=True,
        shuffle=True)

    #Build the model------------------------------

    if model_name == 'non_local':
        log.write('Building non_local Model........')
        net = non_local_network().cuda()
      
    get_parameter_number(net)
    
    try:
        encoder_weight_file = 'se_resnext50_32x4d-a260b3a4.pth'
        net.load_pretrain(encoder_weight_file)
    except:
        pass
    #---------------------------------------------
    if pretrained:
        net.load_state_dict(torch.load(weight_file))

    #bulid the optimizer
    schduler = None  #LR = StepLR([ (0, 0.01),  (200, 0.001),  (300, -1)])
    lr = 0.005
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()),
                          lr=lr, momentum=0.9, weight_decay=0.0001)

   # optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()),
    #                      lr=lr,)

    opt = SWA(optimizer)

    # start training here! #################################################
    log.write('** start training here! **\n')
    best_dice = 0
    best_loss = 1
    best_jac = 0
    
    cosine_learning_schule = []

    for epoch in range(epochs):
        cos_inner = np.pi * (epoch % epochs)  # t - 1 is used when t has 1-based indexing.
        cos_inner /= epochs
        cos_out = np.cos(cos_inner) + 1
        final_lr = float(lr / 2 * cos_out)
        cosine_learning_schule.append(final_lr)

    start_time = time.time()
    for epoch in range(epochs):
        swa_lr = cosine_learning_schule[epoch]
        adjust_learning_rate(opt,swa_lr)
        # train mode
        net.set_mode('train')
        train_loss = 0
        train_dice = 0  #focal_loss
        train_Jac  = 0
        
        train_mixed_loss = 0
        train_mixed_dice = 0
        train_mixed_jac  = 0
        for index, (image, mask) in enumerate(train_dataloader):
            image = image.cuda()
            mask  = mask.cuda()
            logit = net(image)
            loss = net.criterion(logit, mask) 
            dice,Jac = net.metric(logit,mask,thresh)        
            opt.zero_grad()
            loss.backward()
            opt.step()

            train_loss += loss.item()
            train_dice += dice.item()
            train_Jac  += Jac.item()

        train_loss = train_loss / (index+1)
        train_dice = train_dice / (index+1)
        train_Jac  = train_Jac  / (index+1)

        log.write('epoch: {}, Train Loss: {:.4f}, Train Dice: {:.4f}, Train Jac: {:.4f}\n'.format(epoch,train_loss,train_dice,train_Jac))
   #     log.write('Mixed epoch: {}, Train Loss: {:.4f}, Train Dice: {:.4f}, Train Jac: {:.4f}\n'.format(epoch,train_mixed_loss,train_mixed_dice,train_mixed_jac))

        #validation mode
        net.set_mode('valid')
        val_loss = 0
        val_dice = 0
        val_Jac = 0
        for index, (image, mask) in enumerate(val_dataloader):
            image = image.cuda()
            #image_pred = image_pred.cuda()
            mask = mask.cuda()
            #in_mask = in_mask.cuda()
            with torch.no_grad():

                        logit = net(image)
                        
                        preds = F.sigmoid(logit)
                        uncertainty = -1.0*torch.sum(preds*torch.log(preds + 1e-6), dim=1, keepdim=True)
                        ua_logit = uncertainty*logit
                        
                        loss = net.criterion(logit, mask) + 0.5*net.criterion(ua_logit,mask)


                        dice,Jac = net.metric(logit,mask,thresh)

            val_loss += loss.item()
            val_dice += dice.item()
            val_Jac  += Jac.item()

        val_loss = val_loss / (index + 1)
        val_dice = val_dice / (index + 1)
        val_Jac  = val_Jac  / (index + 1)

        log.write('epoch: {}, Valid Loss: {:.4f}, Valid Dice: {:.4f}, Valid Jac: {:.4f}\n'.format(epoch, val_loss, val_dice,val_Jac))


        #save the model's weight
        if val_dice > best_dice:
            best_dice = val_dice
            torch.save(net.state_dict(),out_dir +'/checkpoint/{}_model.pth'.format('best_dice'))
            log.write('Current Best Dice is {}'.format(best_dice))
            #print('Current Best Dice is {}'.format(best_dice))
#        if val_loss < best_loss:
#            best_loss = val_loss
#            torch.save(net.state_dict(),out_dir +'/checkpoint/{}_model.pth'.format('best_loss'))
#            log.write('Current Best Loss is {}'.format(best_loss))
            #print('Current Best Loss is {}'.format(best_loss))
#        if val_Jac > best_jac:
#            best_jac = val_Jac
#            torch.save(net.state_dict(),out_dir +'/checkpoint/{}_model.pth'.format('best_jac'))
#            log.write('Current Best JAC is {}'.format(best_jac))
            #print('Current Best Loss is {}'.format(best_loss))
        if epoch > (epochs-swa_epoch) and epoch % 1 == 0:
            opt.update_swa()
            log.write('SWA Epoch: {}'.format(epoch))
            #torch.save(net.state_dict(),out_dir +'/checkpoint/{}_model.pth'.format(epoch))
            #print('SWA Epoch: {}'.format(epoch))
        
    
    end_time = time.time()
    opt.swap_swa_sgd()
    torch.save(net.state_dict(),out_dir +'/checkpoint/{}_model.pth'.format('SWA'))
    total_time = (end_time - start_time)/3600
    log.write("Total time: {}h".format(total_time))           
    log.close()
    end_time = time.time()



In [ ]:
if __name__ == '__main__':
    shape = (128,128)
    mode = 'train'
    swa_epoch = 5
    pretrained = False
    weight_file = './'
    batch_size = 8
    epochs = 100
    dataset = 'old'
    model_name = 'non_local'
    out_dir = '{}_model_{}'.format(model_name,dataset)
    edge_model = False
    num_class = 1
    dataset_name = 'aiji'
    NFN_mode = False
    SKN_module = False
    kernel_three = False
    if mode == 'train':          
        weight_file = None
        run_my_train(out_dir,shape,swa_epoch,pretrained,weight_file,
                     batch_size,epochs,model_name,edge_model,num_class,dataset,NFN_mode,SKN_module,kernel_three)



In [ ]:
if __name__ == '__main__':
    shape = (128,128)
    mode = 'train'
    swa_epoch = 5
    pretrained = False
    weight_file = './'
    batch_size = 8
    epochs = 100
    dataset = 'new'
    model_name = 'non_local'
    out_dir = '{}_model_{}'.format(model_name,dataset)
    edge_model = False
    num_class = 1
    dataset_name = 'aiji'
    NFN_mode = False
    SKN_module = False
    kernel_three = False
    if mode == 'train':          
        weight_file = None
        run_my_train(out_dir,shape,swa_epoch,pretrained,weight_file,
                     batch_size,epochs,model_name,edge_model,num_class,dataset,NFN_mode,SKN_module,kernel_three)



In [114]:
def run_my_test(test_img_path,weight_file,output_dir,shape,model_name,edge_model,
                num_class=1,uncertainty_path='./uncertainty/',multi_class=False,NFN_model=False,TTA=False,size=224,jigsaw=False,
                SKN_module=False,kernel_three=False,dataset_name=None):

    if model_name == 'non_local':
       # log.write('Building non_local Model........')
        net = non_local_network().cuda()

    get_parameter_number(net)
    os.makedirs(uncertainty_path,exist_ok=True)
    os.makedirs(output_dir,exist_ok=True)
    net.set_mode('valid')
    print('Loading Weight File: {}'.format(weight_file))

    try:
        net.load_state_dict(torch.load(weight_file))

    except:

        load_pretrain_file(net, weight_file, skip)

    print('Done!')
    print('Start Testing')
    #start_time = time.time()
    
    for img_name in tqdm(test_image_list):
        img = cv2.imread(test_img_path+img_name)
        img = cv2.resize(img,shape)

        if TTA:            
            img_hf = cv2.flip(img, 0)
            img_vf = cv2.flip(img, 1)
            img_vhf = cv2.flip(img, -1)
            img_total = np.array([img, img_hf, img_vf,img_vhf])


            img_tensor = torch.from_numpy(np.transpose(img_total,[0,3,1,2]).astype(np.float32))/255

            if edge_model:
                logit,in_logit = net(torch.autograd.Variable(img_tensor).cuda())
                logit = (logit+1-in_logit)/2
            else:
                logit = net(torch.autograd.Variable(img_tensor).cuda())

            index = 0
            pred_total = F.sigmoid(logit)
            pred_total = pred_total.cpu().data.numpy()


            pred_0_0 = pred_total[0,index,:,:]
            pred_1_0 = cv2.flip(pred_total[1,index,:,:],0)
            pred_2_0 = cv2.flip(pred_total[2, index, :, :], 1)
            pred_3_0 = cv2.flip(pred_total[3, index, :, :], -1)

            pred_0 = pred_0_0
            pred_1 = pred_1_0
            pred_2 = pred_2_0
            pred_3 = pred_3_0

            final_pred = 255*(pred_0+pred_1+pred_2+pred_3)/4

        else:
            img_total = np.expand_dims(img,0)
            img_tensor = torch.from_numpy(np.transpose(img_total,[0,3,1,2]).astype(np.float32))/255

            if edge_model:
                logit,in_logit = net(torch.autograd.Variable(img_tensor).cuda())
                logit = (logit+1-in_logit)/2
               # preds = F.sigmoid(logit)
               # uncertainty = -1.0*torch.sum(preds*torch.log(preds + 1e-6), dim=1, keepdim=True)
               # ua_logit = uncertainty*logit
                        

            else:
                logit = net(torch.autograd.Variable(img_tensor).cuda())  
               # preds = F.sigmoid(logit)
               # uncertainty = -1.0*torch.sum(preds*torch.log(preds + 1e-6), dim=1, keepdim=True)
              #  ua_logit = uncertainty*logit
                        
            #logit = ua_logit
            
            index = 0
            pred_total = F.sigmoid(logit)
            pred_total = pred_total.cpu().data.numpy()
            
            final_pred = pred_total[0,index,:,:]*255


        cv2.imwrite(output_dir+img_name,final_pred)